In [1]:
import torch
import numpy as np
from globalbiopak.linop import *
from globalbiopak.phaseretrieval import ForwardPhaseRetrieval
import matplotlib.pyplot as plt

In [2]:
d = 200
n = 1000
x = torch.randn(d, dtype=torch.complex64)
A = torch.randn(n, d, dtype=torch.complex64)
op = LinOpMatrix(A)
forward = ForwardPhaseRetrieval(op)
y = forward.apply(x)

# Gradient descent

In [3]:
n_iter = 100
step = 1e-3
xest = torch.randn(d, dtype=torch.complex64, requires_grad=True)
optimizer = torch.optim.SGD({xest}, lr=step, momentum=0.1, nesterov=True)

for i_iter in range(n_iter):
    optimizer.zero_grad()
    yest = forward.apply(xest)
    loss = torch.norm(yest-y)
    loss.backward()
    optimizer.step()

with torch.no_grad():
    print("Final correlation (close to 1 means successful recovery):")
    print(torch.abs(xest.T.conj() @ x) / \
        torch.sqrt(torch.sum(torch.abs(x)**2)) / torch.sqrt(torch.sum(torch.abs(xest)**2)))

Final correlation (close to 1 means successful recovery):
tensor(0.5725)


# Spectral method

In [4]:
xest = forward.spectralinit(y)
with torch.no_grad():
    print("Correlation after spectral method (close to 1 means successful recovery):")
    print(torch.abs(xest.T.conj() @ x) / \
        torch.sqrt(torch.sum(torch.abs(x)**2)) / torch.sqrt(torch.sum(torch.abs(xest)**2)))

Correlation after spectral method (close to 1 means successful recovery):
tensor(0.8476)


# Gradient descent after spectral initialization

In [5]:
xest.requires_grad = True
optimizer = torch.optim.SGD({xest}, lr=step, momentum=0.1, nesterov=True)

for i_iter in range(n_iter):
    optimizer.zero_grad()
    yest = forward.apply(xest)
    loss = torch.norm(yest-y)
    loss.backward()
    optimizer.step()

with torch.no_grad():
    print("Final correlation (close to 1 means successful recovery):")
    print(torch.abs(xest.T.conj() @ x) / \
        torch.sqrt(torch.sum(torch.abs(x)**2)) / torch.sqrt(torch.sum(torch.abs(xest)**2)))

Final correlation (close to 1 means successful recovery):
tensor(0.9970)
